## Event Planning Automation
In this project, we will automate event planning workflow using 3 Agents :

1. Venue Coordinator
2. Logistics Manager
3. Marketing and Communications Agent

These 3 Agents will perform the following tasks
Venue Task (done by Venue Coordinator)
Logistics Task (done by Logistics Manager) - Async Task
Marketing Task (done by Marketing and Communications Agent) - Async Task
Venue Task will generate a json file venue_details.json using a Pydantic model VenueDetails

Venue Task and Logistics Task will wait for human input to asses and verify the output before finalizing it

In [ ]:
import os
os.environ["OPENAI_MODEL_NAME"] = "gpt-4-turbo"
os.environ["OPENAI_API_KEY"] = "open_api_key"
os.environ["SERPER_API_KEY"] = "API_Key"

In [4]:
from crewai import Agent ,Task,Crew

## Step 03. Creating Agents
Creating 3 agents

1. Venue Coordinator
2. Logistics Manager
3. Marketing and Communications Agent

In [1]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

c:\Users\Dell\AppData\Local\anaconda3\envs\mycrewai\Lib\site-packages\pydantic\_internal\_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [6]:
## Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [ ]:
## Agent2: Logistic manager

logisitics_manager = Agent(
    role="Logistics Manager",
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [9]:
## Agent 3: Marketing and Communication Agent


marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

## Creating VenueDetails Pydantic Model
Create a class VenueDetails using Pydantic BaseModel.
Agents will populate this object with information about different venues by creating different instances of it.

In [10]:
from pydantic import BaseModel

In [11]:
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
By using output_json, you can specify the structure of the output you want.
By using output_file, you can get your output in a file.
By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [14]:
# Task 1: Venue Task

venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    output_json=VenueDetails,
    output_file="venue_details.json",  # Outputs the venue details as a JSON file
    agent=venue_coordinator,
    human_input=True
)

In [21]:
## Logistic Task 

logsitics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    agent=logisitics_manager,
    human_input=True,
    async_execution=False,
)

In [22]:
## Task 3: Marketing Task

marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    agent=marketing_communications_agent,
    async_execution=True,
    output_file="marketing_report.md" # Outputs the report as a text file
)

In [23]:
crew = Crew(
    agents=[venue_coordinator, logisitics_manager, marketing_communications_agent],
    tasks=[venue_task, logsitics_task, marketing_task],
    verbose=True
)


In [25]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [ ]:
results = crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0f4ef32a-f332-4812-bd73-b006588ddc5a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 06f8c6aa-5b07-44ba-b20e-317b2f3d481c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 06f8c6aa-5b07-44ba-b20e-317b2f3d481c
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


🤖 Agent: Venue Coordinator
    Status: In Progress
└── 🧠 Thinking...

...

🤖 Agent: Venue Coordinator
    Status: In Progress



# Agent: Venue Coordinator
## Thought: Now, I will extract details from one of the promising venue links identified from the search results. I will read the content from the Marriott Marquis website as it looks like a venue used for similar conferences.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://worldagritechusa.com/venue/\"}"
## Tool Output: 

World Agri-Tech Innovation Summit, San Francisco: Venue
Skip to content
Contact Us
Home
About
About Us
Advisory Board
Who You’ll Meet
Venue
Upcoming Summits
FAQs
2025 Highlights
Speakers
Partners
Start-Ups
Program
Main Stage
Breakout Sessions
Start-Up Arena
Women’s Breakfast
1-1 Meetings
AI in Agriculture Forum
XPRIZE Workshop
Ginkgo Bioworks Tour
News
News
Newsletter Sign Up
YOUR TICKET
PLATFORM LOGIN
CLOSE
Contact Us
If you have any questions please contact:
Marion Grimes 
Senior Business Development Manager 
Rethink Events 
T: +44 (0)1273 789989
marion.grimes@rethinkevents.com
Rethink Events Ltd, 
1st F

🤖 Agent: Venue Coordinator
    Status: In Progress
└── 🧠 Thinking...

🤖 Agent: Venue Coordinator
    Status: In Progress



# Agent: Venue Coordinator
## Final Answer: 
{
  "name": "Marriott Marquis",
  "address": "780 Mission Street, San Francisco, CA 94103",
  "capacity": 1000,
  "booking_status": "Available"
}


 ## Final Result: {
  "name": "Marriott Marquis",
  "address": "780 Mission Street, San Francisco, CA 94103",
  "capacity": 1000,
  "booking_status": "Available"
}
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====

